In [ ]:
# API and Gradio Demo for Text Summarization    

In [ ]:
#importing the required libraries
import gradio as gr
import requests
import json
import joblib
import numpy as np
import pandas as pd
from transformers import pipeline
from fastapi import FastAPI
from pydantic import BaseModel  


In [ ]:
model = joblib.load('text_summarization_model.pkl')